In [115]:
import os
import warnings
import pandas as pd
import numpy as np
import re 
import math 

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, regexp, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


warnings.filterwarnings('ignore')

In [81]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

# need to only download only once
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chantal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/chantal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Stats

Collect all the datasets

In [82]:
reviews = {}

# assuming naming follows 'type' + '_complete.csv' structure 
for f in os.listdir('data/'):
    if not f.startswith('.'):
        key = re.split(r'_', f)
        reviews[key[0]] = f
        
reviews

{'Scaling': 'Scaling_complete.csv',
 'Rehab': 'Rehab_complete.csv',
 'WASH': 'WASH_complete.csv',
 'ADIPP': 'ADIPP_complete.csv',
 'NCDS': 'NCDS_complete.csv',
 'VitaminD': 'VitaminD_complete.csv'}

In [83]:
PATH = os.path.abspath('data')

for key, dataset in reviews.items():
    reviews[key] = pd.read_csv(os.path.join(PATH, dataset), encoding='latin1').fillna(' ')

In [84]:
reviews['Scaling'].head()

,Title,Authors,Abstract,Published Year,Published Month,Journal,Volume,Issue,Pages,Accession Number,DOI,Ref,Covidence #,Study,Notes,Tags,Inclusion
0,Is this scaling nonlinear? [arXiv],"Leitao, J.C.; Miotto, J.M.; Gerlach, M.; Altma...",One of the most celebrated findings in complex...,2016,,arXiv,,,11-pp.,,,,#9879,Leitao 2016,Takhliq Amir (2019-07-19 07:24:56)(Select): I ...,,0
1,Scaling of foreign attractiveness for countrie...,"Bojic, Iva; Belyi, Alexander; Ratti, Carlo; So...","People's behavior on online social networks, w...",2016,,Applied Geography,73,,47-52,,10.1016/j.apgeog.2016.06.006,,#4381,Bojic 2016,Takhliq Amir (2019-07-18 13:50:05)(Select): Th...,,0
2,Coastal vs inland sensitivity to desertificati...,"Salvati, Luca; Smiraglia, Daniela; Bajocco, So...",The present study assesses the spatial distrib...,2015,,Rendiconti Lincei,26,,571-576,,10.1007/s12210-014-0339-4,,#6536,Salvati 2015,,,0
3,Earthquake hazard and risk assessment based on...,"Kossobokov, V.G.; Nekrasova, A.K.",We apply the general concept of seismic risk a...,2018,,Natural Hazards,93,3,1435-49,,10.1007/s11069-018-3359-z,,#9852,Kossobokov 2018,,,0
4,Dynamic evaluation of seismic hazard and risks...,"Kossobokov, Vladimir G.; Nekrasova, Anastasia",We continue applying the general concept of se...,2016,,American Geophysical Union Fall Meeting,2016,,,,,,#9495,Kossobokov 2016,,,0


Keep only relevant columns: Title, Abstract, Notes and Inclusion

In [85]:
to_keep = ['Title', 'Abstract', 'Notes', 'Inclusion']

for key, dataset in reviews.items():
    reviews[key] = dataset[to_keep]

In [86]:
reviews['Scaling'].head()

,Title,Abstract,Notes,Inclusion
0,Is this scaling nonlinear? [arXiv],One of the most celebrated findings in complex...,Takhliq Amir (2019-07-19 07:24:56)(Select): I ...,0
1,Scaling of foreign attractiveness for countrie...,"People's behavior on online social networks, w...",Takhliq Amir (2019-07-18 13:50:05)(Select): Th...,0
2,Coastal vs inland sensitivity to desertificati...,The present study assesses the spatial distrib...,,0
3,Earthquake hazard and risk assessment based on...,We apply the general concept of seismic risk a...,,0
4,Dynamic evaluation of seismic hazard and risks...,We continue applying the general concept of se...,,0


Concatenate Title and Abstract fields

In [87]:
for key, dataset in reviews.items():
    dataset['All_Text'] = dataset.apply(lambda x: f"{x['Title']} {x['Abstract']}",
                                        axis = 1)

Clean up and preprocess text: remove special characters, punctuation, tokenize, lemmatize, remove any repeated information (e.g., headings), replace NaNs with 0s

In [100]:
# TODO: Do we want to remove numbers and special characters (e.g., other languages??)
def clean_text(s):
    s = s.str.lower()                         # put to lowercase for homogeneity    
    s = s.str.replace(r'_', ' ')              # remove underscores from the notes
    s = s.str.replace(r'\W', ' ')             # remove punctutation
    stop = set(stopwords.words('english'))    # define stop words
    lemmatizer = WordNetLemmatizer()          # lemmatize - a lot of repeat words
    return s.apply(lambda x: [lemmatizer.lemmatize(word, 'v') 
                              for word in x.split() 
                              if word not in stop]) # remove stopwords

In [89]:
for key, dataset in reviews.items():
    dataset[['All_Text']] = dataset[['All_Text']].apply(lambda x: clean_text(x))

In [90]:
# reviews['Scaling'][['All_Text']].apply(lambda x: clean_text(x))

reviews['Scaling']

,Title,Abstract,Notes,Inclusion,All_Text
0,Is this scaling nonlinear? [arXiv],One of the most celebrated findings in complex...,Takhliq Amir (2019-07-19 07:24:56)(Select): I ...,0,"[scale, nonlinear, arxiv, one, celebrate, find..."
1,Scaling of foreign attractiveness for countrie...,"People's behavior on online social networks, w...",Takhliq Amir (2019-07-18 13:50:05)(Select): Th...,0,"[scale, foreign, attractiveness, countries, st..."
2,Coastal vs inland sensitivity to desertificati...,The present study assesses the spatial distrib...,,0,"[coastal, vs, inland, sensitivity, desertifica..."
3,Earthquake hazard and risk assessment based on...,We apply the general concept of seismic risk a...,,0,"[earthquake, hazard, risk, assessment, base, u..."
4,Dynamic evaluation of seismic hazard and risks...,We continue applying the general concept of se...,,0,"[dynamic, evaluation, seismic, hazard, risk, b..."
...,...,...,...,...,...
10686,Scale-Adjusted Metrics for Predicting the Evol...,More than a half of world population is now li...,,1,"[scale, adjust, metrics, predict, evolution, u..."
10687,Empirical analysis on the connection between p...,We report on the existing connection between p...,,1,"[empirical, analysis, connection, power, law, ..."
10688,Positive and negative feedbacks and free-scale...,Depopulation of rural areas is a widespread ph...,Camille Byfield (2019-02-01 05:12:51)(Select):...,1,"[positive, negative, feedbacks, free, scale, p..."
10689,Growth in urban extent and allometric analysis...,Urban area expansion is happening at much fast...,Jean-Luc Kortenaar (2019-07-17 05:49:44)(Selec...,1,"[growth, urban, extent, allometric, analysis, ..."


In [91]:
# average number of tokens, len of abstracts, num notes, label nums
stats = {}

for key, dataset in reviews.items(): 
    avg = np.mean(dataset['All_Text'].apply(lambda x: x.__len__()))
    num = dataset['All_Text'].count()
    num_notes = dataset['Notes'].count() # counts non-NaNs only
    label_counts = dataset['Inclusion'].value_counts()
    
    stats[key] = {'average token length': math.trunc(avg),
                  'total entries': num,
                  'total number of notes': num_notes,
                  'class labels [0]': label_counts[0],
                  'class labels [1]': label_counts[1]}

In [92]:
from pprint import pprint
pprint(stats)

{'ADIPP': {'average token length': 173,
           'class labels [0]': 44402,
           'class labels [1]': 4990,
           'total entries': 49398,
           'total number of notes': 49398},
 'NCDS': {'average token length': 166,
          'class labels [0]': 17883,
          'class labels [1]': 193,
          'total entries': 18078,
          'total number of notes': 18078},
 'Rehab': {'average token length': 155,
           'class labels [0]': 12819,
           'class labels [1]': 220,
           'total entries': 13042,
           'total number of notes': 13042},
 'Scaling': {'average token length': 171,
             'class labels [0]': 10460,
             'class labels [1]': 231,
             'total entries': 10691,
             'total number of notes': 10691},
 'VitaminD': {'average token length': 196,
              'class labels [0]': 1368,
              'class labels [1]': 80,
              'total entries': 1448,
              'total number of notes': 1448},
 'WASH': {'average

# Keyword Extraction

TF-IDF

In [111]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]= score_vals[idx]
    
    return results

In [119]:
# create list of stopwords that also remove pronouns etc. 
stop = list(stopwords.words('english'))

In [124]:
#treating ea. abstract in ea. review as a document

for key, review in reviews.items():
    print('\n******** '+key+' ********')
    corpus = review['Abstract']
    
    cv = CountVectorizer(max_df=0.85, stop_words=stop, max_features=1000)
    word_count_vector = cv.fit_transform(corpus)
    
    tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
  

    feature_names = cv.get_feature_names()
    
    # change this line to whatever document you want to extract keywords from!!
    doc = review['Abstract'].iloc[0]


    tf_idf_vector = tfidf_transformer.transform(cv.transform([doc]))
    sorted_items = sort_coo(tf_idf_vector.tocoo())

    keywords = extract_topn_from_vector(feature_names,sorted_items,10)

    # now print the results
    print("\n===== Doc =====")
    print(doc)
    print("\n=== Keywords ===")
    for k in keywords:
        print(k,keywords[k])

 


******** Scaling ********

===== Doc =====
One of the most celebrated findings in complex systems in the last decade is that different indexes y (e.g., patents) scale nonlinearly with the population x of the cities in which they appear, i.e., y ~ x,   1. More recently, the generality of this finding has been questioned in studies using new databases and different definitions of city boundaries. In this paper we investigate the existence of nonlinear scaling using a probabilistic framework in which fluctuations are accounted explicitly. In particular, we show that this allows not only to (a) estimate  and confidence intervals, but also to (b) quantify the evidence in favor of   1 and (c) test the hypothesis that the observations are compatible with the nonlinear scaling. We employ this framework to compare 5 different models to 15 different datasets and we find that the answers to points (a)-(c) crucially depend on the fluctuations contained in the data, on how they are modeled, and on

# Sentiment Analysis (Brief) 

Is there a link between sentiment and class label = 1?